## Exercise 1

In [1]:
import numpy as np

In [4]:
A = np.array([3.06, 500, 6])
B = np.array([2.68, 320, 4])
C = np.array([2.92, 640, 6])

True


In [17]:
def scale_vector(v, f):
    if v.shape != f.shape:
        return v
    return v * f

def get_cos_dist(v1, v2):
    dist = v1.dot(v2) / np.linalg.norm(v1) * np.linalg.norm(v2)
    print(dist)

In [18]:
# a)

factors = np.array([1, 1, 1])

scaledA = scale_vector(A, factors)
scaledB = scale_vector(B, factors)
scaledC = scale_vector(C, factors)

get_cos_dist(scaledA, scaledB)
get_cos_dist(scaledA, scaledC)
get_cos_dist(scaledB, scaledC)

102422.90926642582
409642.61873518024
409639.5506017164


In [19]:
# b)

factors = np.array([1, 0.01, 0.5])

scaledA = scale_vector(A, factors)
scaledB = scale_vector(B, factors)
scaledC = scale_vector(C, factors)

get_cos_dist(scaledA, scaledB)
get_cos_dist(scaledA, scaledC)
get_cos_dist(scaledB, scaledC)

21.22705985718422
57.99246564438056
56.68372761689134


In [23]:
# c)

factors = np.array([1 / np.mean(A), 1 / np.mean(B), 1 / np.mean(C)])

scaledA = scale_vector(A, factors)
scaledB = scale_vector(B, factors)
scaledC = scale_vector(C, factors)

get_cos_dist(scaledA, scaledB)
get_cos_dist(scaledA, scaledC)
get_cos_dist(scaledB, scaledC)

8.63627862320873
34.543804248192586
34.54370554019432


## Exercise 2

### How can a competitor - in principle - try to steal the valuable data for recommendations on this website?

The attacker could create fake accounts which like certain items where the competitor does not have much data about. The given recommendations can be then collected and possibly exploited.

### Does this work better when the web shop implemented a content-based or a collaborative filtering system?

Content-based systems are problematic for attackers, as they are not based on other user data. So the recommdations are only based on attackers input, which is not useful. Collaborative filtering systems (user-user) is preferred as their recommendations are based on other users' ratings.

### What data would the competitor be able to infer?

The competitor cannot infer any further data from the recommendations. He cannot infer which features the similar products share aswell as the ratings by the similar users. The whole similarity matrix is unknown.

### Would this technique have an impact on the recommendation system, i.e. would this attack create a bias on the data?

Only for Collaborative Filtering (user-user) an attack would create a bias on the data. This requires the attacker to create enough accounts with enough ratings. The reason for that is that the recommender system evaluates all similar users, which could be fake accounts.

### Why is this attack probably not viable in any case?

An attacker does not gain any insight into the user or item data. He will solely be presented with recommendations, which even might be biased and thus not representative.

## Exercise 5

### a) We store the artistid aliases as a dictionary, as it is reasonably small. This makes it efficient to update the utility matrix with the correct artist ids.


In [106]:
import findspark
findspark.init('/usr/local/spark')

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import *

spark = SparkSession.builder.master('local').appName('lab2').getOrCreate()
sc = spark.sparkContext

In [107]:
origSchema = StructType([
    StructField('userid', IntegerType(), True),
    StructField('artistid', IntegerType(), True),
    StructField('playcount', IntegerType(), True)
])

origDF = spark.read.option('delimiter', ' ').schema(origSchema).csv('user_artist_data_small.txt')
origDF.show(2)
origDF.count()

+-------+--------+---------+
| userid|artistid|playcount|
+-------+--------+---------+
|1059637| 1000010|      238|
|1059637| 1000049|        1|
+-------+--------+---------+
only showing top 2 rows



49481

In [127]:
aliasDict = {}
with open('artist_alias_small.txt') as f:
    for line in f:
        (key, val) = line.split()
        aliasDict[int(key)] = int(val)


def get_alias(key):
    ret = key
    if key in aliasDict.keys():
        ret = aliasDict[key]
    return ret

get_alias_udf = F.udf(get_alias, IntegerType())

In [143]:
correctedDF = origDF.withColumn(
    'artistid',
    get_alias_udf(origDF['artistid'])
)

correctedDF.show(2)

+-------+--------+---------+
| userid|artistid|playcount|
+-------+--------+---------+
|1059637| 1000010|      238|
|1059637| 1000049|        1|
+-------+--------+---------+
only showing top 2 rows



### b)

In [167]:
allArtistsDF = correctedDF.select('artistid').distinct().sort(correctedDF.artistid.asc())
test = correctedDF.where(correctedDF.userid == 1059637).sort(correctedDF.artistid.asc())
test.select(avg(test.playcount)).collect()
#test.join(allArtistsDF, test.artistid == allArtistsDF.artistid, 'right').drop('userid').drop('artistid').withColumn('playcount', F.when(F.col('playcount').isNull(), 0).otherwise(F.col('playcount'))).show()

    

[Row(avg(playcount)=1878.5849582172702)]

In [172]:
def get_sim(userid1, userid2, alldf, artistsdf):
    def get_r(userid):
        r = alldf.where(df['userid'] == userid)
        avg = r.select(avg(r['playcount'])).collect()
        # Normalize r by subtracting average
        r = r.select(r['playcount'] - avg)
        # Format r, such that all artist ids are included
        r = r.sort(alldf['artistid'].asc()) \
            .join(artistsdf, alldf['artistid'] == artistsdf['artistid'], 'right') \
            .drop('userid') \
            .drop('artistid') \
            .withColumn(
                'playcount',
                F.when(
                    F.col('playcount').isNull(), 0
                ).otherwise(F.col('playcount'))
            )
        return r
            
    r1 = get_r(userid1)
    r2 = get_r(userid2)